## **ECE 763 Project 3**
#**Sahil Deshpande (200375597)**
#**Shivani Shendre (200375416)**

#Generating Face and Non-Face Data from the FDDB Dataset

Importing Libraries

In [ ]:
import cv2
import numpy as np

Setting data paths

In [ ]:
imagePath = "originalPics/"
#Test Path
test_path = 'FDDB-folds\FDDB-folds\FDDB-fold-10-ellipseList.txt'

#Combining Text files for training data
filenames = ['FDDB-folds\FDDB-folds\FDDB-fold-04-ellipseList.txt','FDDB-folds\FDDB-folds\FDDB-fold-05-ellipseList.txt', 'FDDB-folds\FDDB-folds\FDDB-fold-06-ellipseList.txt','FDDB-folds\FDDB-folds\FDDB-fold-07-ellipseList.txt','FDDB-folds\FDDB-folds\FDDB-fold-08-ellipseList.txt','FDDB-folds\FDDB-folds\FDDB-fold-09-ellipseList.txt']
with open('FDDB-folds\FDDB-folds\combined', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
#Training Path
training_path = 'FDDB-folds\FDDB-folds\Combined'

#Define image size required
img_size = 20

Creating the Dataset

In [ ]:

def data_creation(filepath,flag):

    fp = open(filepath)
    line = fp.readline()
    cnt = 1

    while (line):
        imgPath = imagePath + line.strip()
        image = cv2.imread(imgPath + ".jpg")
        line = fp.readline()

        if(line.strip() == '1'):
            line = fp.readline()

            words = []
            for i in line.split():
                words.append(i)

            major_rad = int(float(words[0]))
            minor_rad = int(float(words[1]))
            angle = float(words[2])
            centre_x = int(float(words[3]))
            centre_y = int(float(words[4]))
            
            ux = major_rad * np.cos(angle);
            uy = major_rad * np.sin(angle);
            vx = minor_rad * np.cos(angle + np.pi/2);
            vy = minor_rad * np.sin(angle + np.pi/2);
            box_halfwidth = np.sqrt(ux*ux + vx*vx);
            box_halfheight = np.sqrt(uy*uy + vy*vy);
            box_ul_corner_x = int(centre_x - box_halfwidth)
            box_ul_corner_y = int(centre_y - box_halfheight)
            box_br_corner_x = int(centre_x + box_halfwidth)
            box_br_corner_y = int(centre_y + box_halfheight)
            
            face1 = image[box_ul_corner_y:box_br_corner_y,box_ul_corner_x:box_br_corner_x]
            nFace1 = image[box_ul_corner_x - 30:box_ul_corner_x - 10,box_ul_corner_y:box_ul_corner_y+20]
            
            
            if(face1.shape[0] > 50 and face1.shape[1] > 50 and nFace1.shape[0] ==img_size  and nFace1.shape[0] ==img_size):
                face1 = cv2.resize(face1,(img_size,img_size))
                if flag == 0:
                    cv2.imwrite("Data Generated/nonface_training/" + str(cnt) +".jpg",nFace1)
                    cv2.imwrite("Data Generated/face_training/" + str(cnt) +".jpg",face1)
                    cnt += 1
                elif flag == 1:
                    cv2.imwrite("Data Generated/nonface_testing/" + str(cnt) + ".jpg", nFace1)
                    cv2.imwrite("Data Generated/face_testing/" + str(cnt) + ".jpg", face1)
                    cnt += 1

            line = fp.readline()

        else:
            word_count = int(line.strip())
            for j in range(word_count):
                next(fp)
            line = fp.readline()
        imgPath = 0